# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.event_detection import detect_rsw_and_rcap

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import get_stops_utils, collect_stop_data

from neural_data_analysis.neural_analysis_tools.glm_tools.glm_fit import general_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained, glm_runner
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func, compare_glm_fit
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.topic_based_neural_analysis.ff_visibility import ff_vis_epochs, vis_design

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import prepare_stop_design

# import decoding
from neural_data_analysis.neural_analysis_tools.decoding_tools.event_decoding import decoding_utils, decoding_analysis, plot_decoding, cmp_decode, load_results
from neural_data_analysis.design_kits.design_by_segment import spike_history
from neural_data_analysis.neural_analysis_tools.decoding_tools.general_decoding import cv_decoding, show_decoding_results
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural.pn_decoding import pn_decoding_model_specs
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import assemble_stop_design
from neural_data_analysis.topic_based_neural_analysis.ff_visibility import decode_vis_utils
from neural_data_analysis.neural_analysis_tools.decoding_tools.decoding_by_topics import decode_vis_pipeline, decode_pn_pipeline, decode_stops_pipeline

import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats
import json

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline


pd.set_option('display.max_colwidth', 200)

# Data from all sessions

In [ ]:
raw_data_dir_name = 'all_monkey_data/raw_monkey_data'
monkey_name = 'monkey_Bruno'

In [ ]:
all_vis_results = show_decoding_results.collect_all_session_decoding_results(
    raw_data_dir_name,
    monkey_name,
    decode_vis_pipeline.FFVisDecodingRunner)

all_vis_results_s = show_decoding_results.collect_all_session_decoding_results(
    raw_data_dir_name,
    monkey_name,
    decode_vis_pipeline.FFVisDecodingRunner,
    shuffle_mode='timeshift_fold')

In [ ]:
all_stops_results = show_decoding_results.collect_all_session_decoding_results(
    raw_data_dir_name,
    monkey_name,
    decode_stops_pipeline.StopDecodingRunner)

all_stops_results_s = show_decoding_results.collect_all_session_decoding_results(
    raw_data_dir_name,
    monkey_name,
    decode_stops_pipeline.StopDecodingRunner,
    shuffle_mode='timeshift_fold',
    )

In [ ]:
all_pn_results = show_decoding_results.collect_all_session_decoding_results(
    raw_data_dir_name,
    monkey_name,
    decode_pn_pipeline.PNDecodingRunner)


all_pn_results_s = show_decoding_results.collect_all_session_decoding_results(
    raw_data_dir_name,
    monkey_name,
    decode_pn_pipeline.PNDecodingRunner,
    shuffle_mode='timeshift_fold')

## Plot

### pn

In [ ]:
reg_summary, clf_summary = show_decoding_results.visualize_decoding_results(
    decoding_results_df=all_pn_results,
    model_name='ridge_strong'
)

In [ ]:
reg_summary, clf_summary = show_decoding_results.visualize_decoding_results(
    decoding_results_df=all_pn_results_s,
    model_name='ridge_strong'
)

### vis

In [ ]:
reg_summary, clf_summary = show_decoding_results.visualize_decoding_results(
    decoding_results_df=all_vis_results,
    model_name='ridge_strong'
)

In [ ]:
reg_summary, clf_summary = show_decoding_results.visualize_decoding_results(
    decoding_results_df=all_vis_results_s,
    model_name='ridge_strong'
)

### stops

In [ ]:
reg_summary, clf_summary = show_decoding_results.visualize_decoding_results(
    decoding_results_df=all_stops_results,
    model_name='ridge_strong'
)

In [ ]:
reg_summary, clf_summary = show_decoding_results.visualize_decoding_results(
    decoding_results_df=all_stops_results_s,
    model_name='ridge_strong'
)

In [ ]:
stop!

# Find best model

## regression

In [ ]:
reg_df = all_stops_results.query("mode == 'regression'")
session_model_table = (
    reg_df
    .pivot_table(
        index=['data_name', 'behav_feature'],
        columns='model_name',
        values='r2_cv'
    )
)

model_means = session_model_table.mean()
model_stds  = session_model_table.std()

model_summary = pd.DataFrame({
    'mean_r2': model_means,
    'std_r2': model_stds
}).sort_values('mean_r2', ascending=False)

model_summary

In [ ]:
wins = (session_model_table
        .idxmax(axis=1)
        .value_counts())

wins

## classification

In [ ]:
reg_df = all_stops_results.query("mode == 'classification'")
session_model_table = (
    reg_df
    .pivot_table(
        index=['data_name', 'behav_feature'],
        columns='model_name',
        values='auc_mean'
    )
)

model_means = session_model_table.mean()
model_stds  = session_model_table.std()

model_summary = pd.DataFrame({
    'auc_mean': model_means,
    'auc_mean': model_stds
}).sort_values('auc_mean', ascending=False)

model_summary

In [ ]:
wins = (session_model_table
        .idxmax(axis=1)
        .value_counts())

wins

# decode stops

## use class

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0220"

In [ ]:
# model_specs = {'ridge': pn_decoding_model_specs.MODEL_SPECS['ridge'],
#               'ridge_strong': pn_decoding_model_specs.MODEL_SPECS['ridge_strong'],
#               }

model_specs = pn_decoding_model_specs.MODEL_SPECS

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0314"

In [ ]:
reload(decode_stops_pipeline)

In [ ]:
runner = decode_stops_pipeline.StopDecodingRunner(
    raw_data_folder_path=raw_data_folder_path,
    bin_width=0.04,
    t_max=0.20,
)

all_results_df = runner.run(n_splits=5,
                            model_specs=model_specs,
                            shuffle_mode='timeshift_fold',
                            )


In [ ]:
runner.stop_feats_to_decode

In [ ]:
all_results_df.sort_values(by='auc_mean', ascending=False).head(20)

In [ ]:
all_results_df.sort_values(by='r2_cv', ascending=False).head(20)

## use script

In [ ]:
!python multiff_analysis/jobs/decoding/decode_stops/scripts/decode_stops_script.py --raw_data_folder_path "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0220"


# decode vis

## use class

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0220"

In [ ]:
runner = decode_vis_pipeline.FFVisDecodingRunner(
    raw_data_folder_path=raw_data_folder_path,
    bin_width=0.04,
    t_max=0.20,
)

all_results_df_s = runner.run(n_splits=5, 
                            model_specs=model_specs,
                            shuffle_mode='timeshift_fold',
                            )

In [ ]:
runner = decode_vis_pipeline.FFVisDecodingRunner(
    raw_data_folder_path=raw_data_folder_path,
    bin_width=0.04,
    t_max=0.20,
)

all_results_df = runner.run(n_splits=5, 
                            model_specs=model_specs
                            )


In [ ]:
all_results_df.sort_values(by='r2_cv', ascending=False).head(20)

In [ ]:
all_results_df.sort_values(by='auc_mean', ascending=False).head(20)

## use script

In [ ]:
!python multiff_analysis/jobs/decoding/decode_vis/scripts/decode_vis_script.py --raw_data_folder_path "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0220"


# decode pn

## use class

In [ ]:
runner = decode_pn_pipeline.PNDecodingRunner(
    raw_data_folder_path=raw_data_folder_path,
    bin_width=0.04,
    t_max=0.20,
)

all_results_df = runner.run(n_splits=5,
                            model_specs=model_specs,
                            )


In [ ]:
all_results_df.sort_values(by='r2_cv', ascending=False).head(20)

In [ ]:
if 'auc_mean' in all_results_df.columns: # note: it's possible that the column is not present, if there is no categorical variable.
    all_results_df.sort_values(by='auc_mean', ascending=False).head(20)

## use script

In [ ]:
!python multiff_analysis/jobs/decoding/decode_pn/scripts/decode_pn_script.py --raw_data_folder_path "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0220"


# decode stop exp

In [ ]:
bin_width = 0.04

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0220"

In [ ]:
global_bins_2d = None

pn, datasets, _ = collect_stop_data.collect_stop_data_func(
    raw_data_folder_path,
    bin_width=bin_width,
)

captures_df, valid_captures_df, filtered_no_capture_stops_df, stops_with_stats = (
    get_stops_utils.prepare_no_capture_and_captures(
        monkey_information=pn.monkey_information,
        closest_stop_to_capture_df=pn.closest_stop_to_capture_df,
        ff_caught_T_new=pn.ff_caught_T_new,
        distance_col='distance_from_ff_to_stop',
    )
)

stops_with_stats['stop_time'] = stops_with_stats['stop_id_start_time']
stops_with_stats['prev_time'] = stops_with_stats['stop_id_end_time'].shift(
    1)
stops_with_stats['next_time'] = stops_with_stats['stop_id_start_time'].shift(
    -1)

new_seg_info = event_binning.make_new_seg_info_for_stop_design(
    stops_with_stats,
    pn.closest_stop_to_capture_df,
    pn.monkey_information,
)

events_with_stats = stops_with_stats[[
    'stop_id',
    'stop_cluster_id',
    'stop_id_start_time',
    'stop_id_end_time',
]].rename(columns={
    'stop_id': 'event_id',
    'stop_cluster_id': 'event_cluster_id',
    'stop_id_start_time': 'event_id_start_time',
    'stop_id_end_time': 'event_id_end_time',
})

stop_binned_spikes, stop_binned_feats, offset_log, stop_meta_used, stop_meta_groups = (
    prepare_stop_design.build_stop_design(
        new_seg_info,
        events_with_stats,
        pn.monkey_information,
        pn.spikes_df,
        pn.ff_dataframe,
        datasets=datasets,
        add_ff_visible_info=True,
        global_bins_2d=global_bins_2d,
    )
)

stop_binned_feats = prepare_stop_design.add_interaction_columns(
    stop_binned_feats
)
stop_binned_feats = prepare_stop_design.scale_binned_feats(
    stop_binned_feats
)

In [ ]:
stop_binned_feats.head(3)

In [ ]:
events_with_stats.head(3)

In [ ]:
# now, from build_stop_design

In [ ]:
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events.prepare_stop_design import add_interaction_columns, scale_binned_feats, add_ff_visible_and_in_memory_info, add_retries_info_to_binned_feats
from typing import Tuple, Dict, List
add_ff_visible_info = True
add_retries_info = True
bin_dt = 0.04


monkey_information = pn.monkey_information
spikes_df = pn.spikes_df
ff_dataframe = pn.ff_dataframe
# -------------------------------------------------------------------------
# 1) Build bins from event windows
# -------------------------------------------------------------------------
bins_2d, meta = event_binning.event_windows_to_bins2d(
    new_seg_info, bin_dt=bin_dt, only_ok=False, global_bins_2d=global_bins_2d
)

# -------------------------------------------------------------------------
# 2) Assign continuous samples to bins
# -------------------------------------------------------------------------
sample_idx, bin_idx_arr, dt_arr, _ = event_binning.build_bin_assignments(
    monkey_information['time'].to_numpy(), bins_2d
)

monkey_sub = monkey_information.iloc[sample_idx].copy()

# -------------------------------------------------------------------------
# 3) Compute exposure and valid bins
# -------------------------------------------------------------------------
_, exposure, used_bins = event_binning.bin_timeseries_weighted(
    monkey_sub['time'].to_numpy(), dt_arr, bin_idx_arr, how='mean'
)

def _agg_feat(col: str) -> np.ndarray:
    vals = monkey_sub[col].to_numpy()
    out, exp_chk, used_bins_chk = event_binning.bin_timeseries_weighted(
        vals, dt_arr, bin_idx_arr, how='mean'
    )
    if not np.allclose(exp_chk, exposure):
        raise ValueError(f'Exposure mismatch while aggregating "{col}"')
    if not np.array_equal(used_bins_chk, used_bins):
        raise ValueError(f'used_bins mismatch while aggregating "{col}"')
    return out

# -------------------------------------------------------------------------
# 4) Aggregate kinematics
# -------------------------------------------------------------------------
KINEMATIC_COLS = ['accel', 'speed', 'ang_speed']
binned_feats = (
    pd.DataFrame({c: _agg_feat(c) for c in KINEMATIC_COLS})
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0.0)
)

mask_used = exposure > 0
pos = used_bins[mask_used]

binned_feats = binned_feats.iloc[mask_used].reset_index(drop=True)

print('meta.shape', meta.shape)

meta_used = (
    meta.set_index('bin')
    .sort_index()
    .loc[pos]
    .reset_index()
)

# -------------------------------------------------------------------------
# 5) Bin spikes per cluster
# -------------------------------------------------------------------------
spike_counts, cluster_ids = event_binning.bin_spikes_by_cluster(
    spikes_df, bins_2d, time_col='time', cluster_col='cluster'
)

binned_spikes = (
    pd.DataFrame(spike_counts[pos, :], columns=cluster_ids)
    .reset_index(drop=True)
)

#########################
# -------------------------------------------------------------------------
# 6) Event- and cluster-level features
# -------------------------------------------------------------------------
X_event_df = stop_design.build_event_design_from_meta(
    meta=meta,
    pos=pos,
    new_seg_info=new_seg_info,
    speed_used=binned_feats['speed'].values,
    include_columns=(
        'basis', 'prepost', 'prepost*speed',
        'captured', 'basis*captured',
        'time_since_prev_event', 'time_to_next_event',
    )
)

cluster_df = cluster_design.build_cluster_features_workflow(
    meta_used[['event_id', 'rel_center']],
    events_with_stats,
    rel_time_col='rel_center',
    winsor_p=0.5,
    use_midbin_progress=True,
    zscore_progress=False,
    zscore_rel_time=True,
)

CLUSTER_FEATS = [
    'is_clustered',
    'event_is_first_in_cluster',
    'prev_gap_s_z',
    'next_gap_s_z',
    'cluster_duration_s_z',
    'cluster_progress_c',
    'cluster_progress_c2',
    'bin_t_from_cluster_start_s_z',
]

def _safe_add_columns(target_df, source_df, cols):
    cols = [c for c in cols if c not in target_df.columns]
    if cols:
        target_df.loc[:, cols] = source_df[cols].to_numpy()

_safe_add_columns(binned_feats, X_event_df, X_event_df.columns)
_safe_add_columns(binned_feats, cluster_df, CLUSTER_FEATS)

# -------------------------------------------------------------------------
# 7) Offset term and timing
# -------------------------------------------------------------------------
offset_log = np.log(np.clip(exposure[mask_used], 1e-12, None))
binned_feats['time_rel_to_event_start'] = meta_used['rel_center'].to_numpy()

# -------------------------------------------------------------------------
# 8) Firefly visibility / memory features
# -------------------------------------------------------------------------
if add_ff_visible_info:
    binned_feats = add_ff_visible_and_in_memory_info(
        binned_feats,
        bins_2d,
        ff_dataframe,
        used_bins,
        max_in_memory_time_since_seen=2,
    )

# -------------------------------------------------------------------------
# 9) Retry-related features
# -------------------------------------------------------------------------
if add_retries_info:
    binned_feats, meta_used = add_retries_info_to_binned_feats(
        binned_feats, new_seg_info, datasets, meta_used
    )

# -------------------------------------------------------------------------
# 10) Feature groups
# -------------------------------------------------------------------------
groups: Dict[str, List[str]] = {}

def _add_group(name: str, cols: List[str]):
    cols = [c for c in cols if c in binned_feats.columns]
    if cols:
        groups[name] = cols

for c in KINEMATIC_COLS:
    _add_group(c, [c])

_add_group(
    'basis',
    [c for c in binned_feats.columns if c.startswith('rcos_') and '*captured' not in c]
)
_add_group(
    'basis*captured',
    [c for c in binned_feats.columns if c.startswith('rcos_') and c.endswith('*captured')]
)

_add_group('prepost', ['prepost'])
_add_group('prepost*speed', ['prepost*speed'])
_add_group('captured', ['captured'])
_add_group('time_since_prev_event', ['time_since_prev_event'])
_add_group('time_to_next_event', ['time_to_next_event'])

_add_group('cluster_flags', ['event_is_first_in_cluster'])
_add_group('cluster_gaps', ['prev_gap_s_z', 'next_gap_s_z'])
_add_group('cluster_duration', ['cluster_duration_s_z'])
_add_group('cluster_progress', ['cluster_progress_c', 'cluster_progress_c2'])
_add_group('cluster_rel_time', ['bin_t_from_cluster_start_s_z'])

_add_group('ff_visible', ['log1p_num_ff_visible', 'k_ff_visible'])
_add_group('ff_in_memory', ['log1p_num_ff_in_memory', 'k_ff_in_memory'])

_add_group(
    'retries',
    [
        'rsw_first', 'rcap_first',
        'rsw_middle', 'rcap_middle',
        'rsw_last', 'rcap_last',
        'one_stop_miss',
        'whether_in_retry_series',
        'miss',
    ],
)

_add_group('time_rel_to_event_start', ['time_rel_to_event_start'])



In [ ]:
cluster_df

In [ ]:
X_event_df

In [ ]:
binned_feats

In [ ]:
X_event_df2 = stop_design.build_event_design_for_decoding(
    meta=meta,
    pos=pos,
    new_seg_info=new_seg_info,
)

In [ ]:
X_event_df2

In [ ]:
stop_binned_spikes, stop_binned_feats, offset_log, stop_meta_used, stop_meta_groups = (
    prepare_stop_design.build_stop_design(
        new_seg_info,
        events_with_stats,
        pn.monkey_information,
        pn.spikes_df,
        pn.ff_dataframe,
        datasets=datasets,
        add_ff_visible_info=True,
        global_bins_2d=global_bins_2d,
        for_decoding=True,
    )
)

In [ ]:
stop_binned_feats.columns

In [ ]:
stop_binned_spikes, stop_binned_feats, offset_log, stop_meta_used, stop_meta_groups = (
    prepare_stop_design.build_stop_design(
        new_seg_info,
        events_with_stats,
        pn.monkey_information,
        pn.spikes_df,
        pn.ff_dataframe,
        datasets=datasets,
        add_ff_visible_info=True,
        global_bins_2d=global_bins_2d,
        for_decoding=False,
    )
)

In [ ]:
stop_binned_feats.columns

In [ ]:
event_t_from_cluster_start_s